In [1]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
import os

# Setup
sagemaker_session = sagemaker.Session()
role = sagemaker.get_execution_role()

# Upload training data
input_s3 = sagemaker_session.upload_data(path='wine_quality_dataset.csv', key_prefix='wine-quality-data')

print(f"Dataset uploaded to: {input_s3}")


sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /home/sagemaker-user/.config/sagemaker/config.yaml
Dataset uploaded to: s3://sagemaker-us-east-1-107984457514/wine-quality-data/wine_quality_dataset.csv


In [6]:
# Define the SKLearn Estimator
sklearn_estimator = SKLearn(
    entry_point='train.py',
    role=role,
    instance_type='ml.m5.large',
    framework_version='0.23-1',
    sagemaker_session=sagemaker_session,
    base_job_name='wine-quality-train',
    hyperparameters={},
    output_path=f's3://{sagemaker_session.default_bucket()}/output'
)

# Start training
sklearn_estimator.fit({'train': input_s3})


INFO:sagemaker.telemetry.telemetry_logging:SageMaker Python SDK will collect telemetry to help us better understand our user's needs, diagnose issues, and deliver additional features.
To opt out of telemetry, please disable via TelemetryOptOut parameter in SDK defaults config. For more information, refer to https://sagemaker.readthedocs.io/en/stable/overview.html#configuring-and-using-defaults-with-the-sagemaker-python-sdk.
INFO:sagemaker:Creating training-job with name: wine-quality-train-2025-05-24-15-33-48-698


2025-05-24 15:33:50 Starting - Starting the training job...
2025-05-24 15:34:04 Starting - Preparing the instances for training...
2025-05-24 15:34:50 Downloading - Downloading the training image.....
2025-05-24 15:35:41 Training - Training image download completed. Training in progress.
2025-05-24 15:35:41 Uploading - Uploading generated training model2025-05-24 15:35:35,329 sagemaker-containers INFO     Imported framework sagemaker_sklearn_container.training
2025-05-24 15:35:35,333 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-24 15:35:35,378 sagemaker_sklearn_container.training INFO     Invoking user training script.
2025-05-24 15:35:35,565 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-24 15:35:35,580 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-05-24 15:35:35,594 sagemaker-training-toolkit INFO     No GPUs detected (normal if no gpus installed)
2025-0

In [7]:
print(sklearn_estimator.latest_training_job.describe()['TrainingJobStatus'])


Completed


In [8]:
model_artifact_path = sklearn_estimator.model_data
print(f"Model saved to: {model_artifact_path}")


Model saved to: s3://sagemaker-us-east-1-107984457514/output/wine-quality-train-2025-05-24-15-33-48-698/output/model.tar.gz


In [11]:
from sagemaker.sklearn.model import SKLearnModel

sklearn_model = SKLearnModel(
    model_data=model_artifact_path,  # Output from training
    role=role,
    entry_point='serve.py',
    framework_version='0.23-1',
    sagemaker_session=sagemaker_session
)

predictor = sklearn_model.deploy(
    instance_type='ml.m5.large',
    initial_instance_count=1
)

endpoint_name = predictor.endpoint_name
print(f"Deployed to endpoint: {endpoint_name}")


INFO:sagemaker:Creating model with name: sagemaker-scikit-learn-2025-05-24-15-48-39-962
INFO:sagemaker:Creating endpoint-config with name sagemaker-scikit-learn-2025-05-24-15-48-41-037
INFO:sagemaker:Creating endpoint with name sagemaker-scikit-learn-2025-05-24-15-48-41-037


--------!Deployed to endpoint: sagemaker-scikit-learn-2025-05-24-15-48-41-037
